In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import CRUD operations for retrieving animal data from MongoDB
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
shelter = AnimalShelter()

# Retrieve all records from MongoDB and populate the table
df = pd.DataFrame.from_records(shelter.read_all()) 

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load Grazioso Salvare’s logo and encode it in base64 for embedding
image_path = "/home/shaynesulliva_snhu/Pictures/GraziosoSalvareLogo.png" 
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Define columns to display in the data table
columns_to_include = ["animal_type", "name", "breed", "sex_upon_outcome", "age_upon_outcome_in_weeks", "outcome_type"]

# Define app layout
app.layout = html.Div(style={'backgroundColor': '#121212', 'color': '#ffffff', 'padding': '20px'}, children=[
    # Centered logo with padding
    html.Div(
        html.A(
            html.Img(
                src=f"data:image/png;base64,{encoded_image}", 
                style={'height': '250px'}
            ),
            href="https://www.snhu.edu",
            target="_blank"
        ),
        style={'textAlign': 'center', 'padding': '10px'}
    ),

    # Unique identifier inside the Dashboard Header
    html.Center([
        html.H1('CS-340 Dashboard', style={'color': '#FF3B3F'}),  # Red title
        html.H3('Created by Shayne Sullivan', style={'fontSize': '18px', 'color': '#E63946'})
    ]),

    html.Hr(style={'borderColor': '#FF3B3F'}),  # Visual separator

    # Interactive Filtering Options
    html.Div(
        style={'display': 'flex', 'alignItems': 'center', 'color': '#ffffff'},
        children=[
            html.Label("Filter by Rescue Type:", style={'fontSize': '16px', 'fontWeight': 'bold', 'margin-right': '15px'}),
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                    {'label': 'All Preferred Breeds', 'value': 'All Preferred Breeds'},  
                    {'label': 'Reset', 'value': 'Reset'}
                ],
                value='Reset',
                labelStyle={'display': 'inline-block', 'margin-right': '15px', 'color': '#FF3B3F'}
            )
        ]
    ),


    html.Hr(style={'borderColor': '#FF3B3F'}),  

    # DataTable to display animal records
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in columns_to_include],
        data=df[columns_to_include].to_dict('records'),
        row_selectable="single",
        selected_rows=[0],
        page_size=10,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",

        # Styling for readability and dark mode consistency
        style_table={'overflowX': 'auto', 'backgroundColor': '#1E1E1E'},
        style_data={'backgroundColor': '#1E1E1E', 'color': '#ffffff'},
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': '#2A2A2A'},
            {'if': {'state': 'selected'}, 'backgroundColor': '#FF3B3F', 'color': 'white'}
        ],
        style_header={'fontWeight': 'bold', 'textAlign': 'center', 'backgroundColor': '#E63946', 'color': 'white'},
        style_cell={'whiteSpace': 'normal', 'height': 'auto', 'minWidth': '100px', 'width': '150px', 'maxWidth': '200px'}
    ),

    html.Br(),
    html.Hr(style={'borderColor': '#FF3B3F'}),  

    # Layout for map and chart
    html.Div(className='row',
        style={'display': 'flex'},
        children=[
            # Map Container (Always on the Left)
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'flex': '1', 'backgroundColor': '#1E1E1E'}  
            ),
            # Chart Container (Always on the Right)
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style={'flex': '1', 'backgroundColor': '#1E1E1E'}  
            )
        ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:  # If no columns are selected, return an empty list
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update the data displayed in the table based on filter selection    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if not filter_type or filter_type == 'Reset':  
        # Return all data if Reset is selected or when the app loads
        return df.to_dict('records')  

    # Define breed categories based on Grazioso Salvare's requirements
    preferred_breeds = {
        'Water Rescue': ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        'Mountain or Wilderness Rescue': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        'Disaster or Individual Tracking': ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
    }

    # Define filtering logic
    queries = {
        'Water Rescue': {
            "breed": {"$in": preferred_breeds['Water Rescue']},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        },
        'Mountain or Wilderness Rescue': {
            "breed": {"$in": preferred_breeds['Mountain or Wilderness Rescue']},
            "sex_upon_outcome": "Intact Male"
        },
        'Disaster or Individual Tracking': {
            "breed": {"$in": preferred_breeds['Disaster or Individual Tracking']},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        },
        'All Preferred Breeds': {
            "breed": {"$in": sum(preferred_breeds.values(), [])}  
        },
        'Other Breeds': {
            "breed": {"$nin": sum(preferred_breeds.values(), [])}  
        }
    }
    
    # Retrieve filtered data
    query = queries.get(filter_type, {})
    df_filtered = pd.DataFrame.from_records(shelter.read(query))

    if df_filtered.empty:
        return []  # If no results, return empty list to clear table

    if "_id" in df_filtered:
        df_filtered.drop(columns=['_id'], inplace=True)

    return df_filtered.to_dict('records')


# Display pie chart based on selected filter
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)
def update_graphs(viewData, filter_type):
    # Only display the chart if a filter is applied
    if not viewData or filter_type == "Reset":
        return [html.Div(
            "Select a filter to display the breed distribution chart.",
            style={'color': 'white', 'textAlign': 'right'}  
        )]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create pie chart visualization
    fig = px.pie(
        dff,
        names='breed',
        title='Breed Distribution',
        color_discrete_sequence=['#FF3B3F', '#E63946', '#D62828', '#BA181B', '#A4161A'],  
    )

    # Update figure layout for dark mode
    fig.update_layout(
        paper_bgcolor='#121212',  
        plot_bgcolor='#121212',   
        font=dict(color="white"),  
        title_font=dict(size=18, color='white'),  
        margin=dict(l=20, r=20, t=50, b=20),  
    )

    return [
        dcc.Graph(
            figure=fig,
            style={'width': '100%', 'height': '500px'}  # Ensures alignment with map
        )
    ]

# Display map markers for filtered animals
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)
def update_map(viewData, filter_type):
     # Only display the map if a filter is applied
    if not viewData or filter_type == "Reset":
        return [html.Div("Select a filter to display the map.", style={'color': 'white'})]

    dff = pd.DataFrame.from_dict(viewData)

    # If no filter is selected (Reset), return an empty map
    if not filter_type or filter_type == "Reset":
        return [dl.Map(style={'width': '800px', 'height': '500px'}, center=[30.75, -97.48], zoom=8, children=[
            dl.TileLayer(id="base-layer-id")
        ])]

    # Generate a list of markers for each animal in the filtered dataset
    markers = []
    for _, row in dff.iterrows():
        lat, lon = row['location_lat'], row['location_long']
        
        # Include all fields except latitude and longitude
        popup_info = []
        for col, value in row.items():
            if col not in ['location_lat', 'location_long']:
                popup_info.append(html.P(f"{col.replace('_', ' ').title()}: {value}"))

        markers.append(
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(f"{row['name']} ({row['breed']})"),  # Tooltip on hover
                dl.Popup([
                    html.Div(
                        children=[
                            html.H1(row['name']),
                            *popup_info  
                        ],
                        style={
                            'maxHeight': '300px',  # Set max height for scroll
                            'overflowY': 'auto',  # Enable vertical scroll
                            'padding': '10px'
                        }
                    )
                ])
            ])
        )

    # Return the updated map with all markers
    return [
        dl.Map(style={'width': '800px', 'height': '500px'},
               center=[dff.iloc[0]['location_lat'], dff.iloc[0]['location_long']],  # Center map on first result
               zoom=10,
               children=[
                   dl.TileLayer(id="base-layer-id"),
                   *markers  
               ])
    ]

app.run_server(debug=True)


✅ Connected to MongoDB successfully!
Dash app running on http://127.0.0.1:13914/
